In [1]:
"""カルマンフィルター(設定例)"""
import sympy
sympy.init_printing()

In [2]:
"""設定例"""
# システム状態はトロッコの位置と速度
x, dx = sympy.symbols('x_{k} \dot{x}_{k}')
xk = sympy.MatrixSymbol('xbm_k',2, 1)
_xk = sympy.Matrix([[x],[dx]])

sympy.Eq(xk,_xk)

       ⎡   x_{k}   ⎤
xbmₖ = ⎢           ⎥
       ⎣\dot{x}_{k}⎦

In [3]:
# 時間変化なしFk=F
Fk = sympy.MatrixSymbol('F_k', 2, 2)
dt = sympy.symbols('{\Delta}t')
F = sympy.Matrix([[1,dt],[0,1]])

# 制御はなしuk＝0
uk = sympy.MatrixSymbol('ubm_k', 2,1)
_uk = sympy.Matrix([[0],[0]])

sympy.Eq(Fk,F),\
sympy.Eq(uk,_uk)

⎛     ⎡1  {\Delta}t⎤         ⎡0⎤⎞
⎜Fₖ = ⎢            ⎥, ubmₖ = ⎢ ⎥⎟
⎝     ⎣0      1    ⎦         ⎣0⎦⎠

In [4]:
# 雑音なしのときのシステム状態遷移
# (運動の第２法則)
sympy.Eq(xk, F*_xk + _uk)

       ⎡\dot{x}_{k}⋅{\Delta}t + x_{k}⎤
xbmₖ = ⎢                             ⎥
       ⎣         \dot{x}_{k}         ⎦

In [5]:
# 雑音によって加速されるものとする
# akは平均0標準偏差σaの正規分布
wk = sympy.MatrixSymbol('wbm_k', 1, 1)
ak, sa = sympy.symbols('a_{k} {\sigma}_a')
_wk = sympy.Matrix([ak])

Qk = sympy.MatrixSymbol('Q_k', 1, 1)
Q = sympy.Matrix([sa**2])

sympy.Eq(wk,_wk),\
sympy.Function('N')(0,Q),\
sympy.Eq(Qk,Q)

⎛                 ⎛   ⎡         2⎤⎞       ⎡         2⎤⎞
⎝wbmₖ = [a_{k}], N⎝0, ⎣{\sigma}ₐ ⎦⎠, Qₖ = ⎣{\sigma}ₐ ⎦⎠

In [6]:
# 雑音によって加速されるものとする
# 時間変化なしG
# dt間に加速度akが与えられる
Gk = sympy.MatrixSymbol('G_k', 2, 1)
G = sympy.Matrix([
    [dt**2*sympy.Rational(1,2)],[dt]])
sympy.Eq(Gk,G)

     ⎡         2⎤
     ⎢{\Delta}t ⎥
     ⎢──────────⎥
Gₖ = ⎢    2     ⎥
     ⎢          ⎥
     ⎣{\Delta}t ⎦

In [7]:
# 雑音ありのときのシステムの状態遷移
# 運動の第2法則より
sympy.Eq(xk, F*_xk+G*_wk)

       ⎡                                       2        ⎤
       ⎢                        a_{k}⋅{\Delta}t         ⎥
       ⎢\dot{x}_{k}⋅{\Delta}t + ──────────────── + x_{k}⎥
xbmₖ = ⎢                               2                ⎥
       ⎢                                                ⎥
       ⎣         \dot{x}_{k} + a_{k}⋅{\Delta}t          ⎦

In [8]:
# 位置を観測する(雑音なし)
# 時間変化なしH
zk = sympy.MatrixSymbol('zbm_k', 1, 1)
_zk = sympy.Matrix([sympy.symbols('z_k')])

Hk = sympy.MatrixSymbol('H_k', 1, 2)
H = sympy.Matrix([[1,0]])

sympy.Eq(zk,_zk),\
sympy.Eq(Hk,H),\
sympy.Eq(_zk,H*_xk)

(zbmₖ = [zₖ], Hₖ = [1  0], [zₖ] = [x_{k}])

In [9]:
# 観測雑音も平均0、標準偏差σzの正規分布
sz = sympy.symbols('{\\sigma}_z')
vk = sympy.MatrixSymbol('vbm_k', 1, 1)
_vk = sympy.Matrix([sympy.symbols('v_k')])
Rk = sympy.MatrixSymbol('R_k',1, 1)
R = sympy.Matrix([sz**2])
                        
sympy.Eq(vk,_vk),\
sympy.Function('N')(0,sz**2),\
sympy.Eq(Rk,R)

⎛              ⎛             2⎞       ⎡          2⎤⎞
⎝vbmₖ = [vₖ], N⎝0, {\sigma}_z ⎠, Rₖ = ⎣{\sigma}_z ⎦⎠

In [10]:
# 位置を観測する(雑音あり)
sympy.Eq(_zk,H*_xk+_vk)

[zₖ] = [vₖ + x_{k}]

In [11]:
E = sympy.Function('E')
cov = sympy.Function('cov')

In [12]:
"""設定例の予測"""
# 時刻k-1で推定したシステム状態の推定値
_xk1_k1_x = sympy.symbols('\hat{x}_{k-1|k-1}')
_xk1_k1_dx = sympy.symbols('\hat{\dot{x}}_{k-1|k-1}')
_xk1_k1 = sympy.Matrix([[_xk1_k1_x],[_xk1_k1_dx]])        

# 時刻k-1の推定値から予測した、時刻kのシステム状態の推定値
_xk_k1_x = sympy.symbols('\hat{x}_{k|k-1}')
_xk_k1_dx = sympy.symbols('\hat{\dot{x}}_{k|k-1}')
_xk_k1 = sympy.Matrix([[_xk_k1_x],[_xk_k1_dx]])        

sympy.Eq(_xk_k1,F*_xk1_k1),\
sympy.Eq(E(_xk-_xk_k1),0)

⎛⎡   \hat{x}_{k|k-1}   ⎤   ⎡\hat{\dot{x}}_{k-1|k-1}⋅{\Delta}t + \hat{x}_{k-1|k
⎜⎢                     ⎥ = ⎢                                                  
⎝⎣\hat{\dot{x}}_{k|k-1}⎦   ⎣               \hat{\dot{x}}_{k-1|k-1}            

-1}⎤   ⎛⎡     -\hat{x}_{k|k-1} + x_{k}      ⎤⎞    ⎞
   ⎥, E⎜⎢                                   ⎥⎟ = 0⎟
   ⎦   ⎝⎣\dot{x}_{k} - \hat{\dot{x}}_{k|k-1}⎦⎠    ⎠

In [13]:
"""設定例の予測(精度)"""
# システム状態の誤差の共分散行列(推定値の精度)
# 時刻k-1で推定したシステム状態の推定値の精度
Pk1_k1 = sympy.MatrixSymbol('P_{k-1|k-1}', 2, 2)
#_Pk1_k1 = sympy.symbols('P^{{0:2}{0:2}}_{k-1|k-1}')
#Pk1_k1 = sympy.Matrix([_Pk1_k1[0:2],_Pk1_k1[2:4]])

# システムモデルと時刻k-1の精度から予測される、時刻kでのシステム状態の精度
Pk_k1 = sympy.MatrixSymbol('P_{k|k-1}', 2, 2)
_Pk_k1 = F*Pk1_k1*F.T + G*Q*G.T

sympy.Eq(Pk_k1,_Pk_k1)
#_Pk_k1[3]

#sympy.Eq(Pk_k1,cov(xk-xk_k1)*sympy.Identity(n)),

            ⎡         4          2           3          2⎤                    
            ⎢{\Delta}t ⋅{\sigma}ₐ   {\Delta}t ⋅{\sigma}ₐ ⎥                    
            ⎢─────────────────────  ─────────────────────⎥                    
            ⎢          4                      2          ⎥   ⎡1  {\Delta}t⎤   
P_{k|k-1} = ⎢                                            ⎥ + ⎢            ⎥⋅P_
            ⎢         3          2                       ⎥   ⎣0      1    ⎦   
            ⎢{\Delta}t ⋅{\sigma}ₐ            2          2⎥                    
            ⎢─────────────────────  {\Delta}t ⋅{\sigma}ₐ ⎥                    
            ⎣          2                                 ⎦                    

                        
                        
                        
          ⎡    1      0⎤
{k-1|k-1}⋅⎢            ⎥
          ⎣{\Delta}t  1⎦
                        
                        
                        

In [14]:
"""更新(観測残差,innovation)"""
# 観測値と、予測したシステム状態の推定値から
# 計算した予測観測値との差
ek = sympy.MatrixSymbol('ebm_k', 1, 1)
_ek = _zk-H*_xk_k1
sympy.Eq(ek,_ek),\
sympy.Eq(E(ek),0)

(ebmₖ = [-\hat{x}_{k|k-1} + zₖ], E(ebmₖ) = 0)

In [15]:
"""更新(観測残差の共分散)"""
Sk = sympy.MatrixSymbol('S_k', 1, 1)
_Sk = R+H*Pk_k1*H.T
sympy.Eq(Sk,_Sk)

     ⎡          2⎤                    ⎡1⎤
Sₖ = ⎣{\sigma}_z ⎦ + [1  0]⋅P_{k|k-1}⋅⎢ ⎥
                                      ⎣0⎦

In [16]:
"""更新(最適カルマンゲイン)"""
Kk = sympy.MatrixSymbol('K_k', 2, 1)
_Kk = _Pk_k1*H.T*_Sk.inverse()
sympy.Eq(Kk, _Kk)

     ⎛⎡         4          2           3          2⎤                          
     ⎜⎢{\Delta}t ⋅{\sigma}ₐ   {\Delta}t ⋅{\sigma}ₐ ⎥                          
     ⎜⎢─────────────────────  ─────────────────────⎥                          
     ⎜⎢          4                      2          ⎥   ⎡1  {\Delta}t⎤         
Kₖ = ⎜⎢                                            ⎥ + ⎢            ⎥⋅P_{k-1|k
     ⎜⎢         3          2                       ⎥   ⎣0      1    ⎦         
     ⎜⎢{\Delta}t ⋅{\sigma}ₐ            2          2⎥                          
     ⎜⎢─────────────────────  {\Delta}t ⋅{\sigma}ₐ ⎥                          
     ⎝⎣          2                                 ⎦                          

                  ⎞                                             
                  ⎟                                             
                  ⎟                                           -1
    ⎡    1      0⎤⎟ ⎡1⎤ ⎛⎡          2⎤                    ⎡1⎤⎞  
-1}⋅⎢            ⎥⎟⋅⎢ ⎥⋅⎜⎣{\

In [17]:
"""観測値によるシステム状態の推定値の更新"""
xk_k = sympy.MatrixSymbol('\hat{x}_{k|k}', 2, 1)
_xk_k = _xk_k1 + _Kk*_ek
sympy.Eq(xk_k, _xk_k)
#sympy.Eq(E(xk-xk_k),0)

                                          ⎛⎡         4          2           3 
                                          ⎜⎢{\Delta}t ⋅{\sigma}ₐ   {\Delta}t ⋅
                                          ⎜⎢─────────────────────  ───────────
                ⎡   \hat{x}_{k|k-1}   ⎤   ⎜⎢          4                      2
\hat{x}_{k|k} = ⎢                     ⎥ + ⎜⎢                                  
                ⎣\hat{\dot{x}}_{k|k-1}⎦   ⎜⎢         3          2             
                                          ⎜⎢{\Delta}t ⋅{\sigma}ₐ            2 
                                          ⎜⎢─────────────────────  {\Delta}t ⋅
                                          ⎝⎣          2                       

         2⎤                                            ⎞                      
{\sigma}ₐ ⎥                                            ⎟                      
──────────⎥                                            ⎟                      
          ⎥   ⎡1  {\Delta}t⎤             ⎡    1    

In [21]:
"""観測値によるシステム状態の推定値の精度の更新"""
Pk_k = sympy.MatrixSymbol('P_{k|k}', 2, 2)


sympy.Eq(Pk_k, (sympy.Identity(2)-_Kk*H)*Pk_k1)
#sympy.Eq(Pk_k,cov(xk-xk_k)*sympy.Identity(n))

          ⎛   ⎛⎡         4          2           3          2⎤                 
          ⎜   ⎜⎢{\Delta}t ⋅{\sigma}ₐ   {\Delta}t ⋅{\sigma}ₐ ⎥                 
          ⎜   ⎜⎢─────────────────────  ─────────────────────⎥                 
          ⎜   ⎜⎢          4                      2          ⎥   ⎡1  {\Delta}t⎤
P_{k|k} = ⎜I -⎜⎢                                            ⎥ + ⎢            ⎥
          ⎜   ⎜⎢         3          2                       ⎥   ⎣0      1    ⎦
          ⎜   ⎜⎢{\Delta}t ⋅{\sigma}ₐ            2          2⎥                 
          ⎜   ⎜⎢─────────────────────  {\Delta}t ⋅{\sigma}ₐ ⎥                 
          ⎝   ⎝⎣          2                                 ⎦                 

                           ⎞                                                  
                           ⎟                                                  
                           ⎟                                           -1     
             ⎡    1      0⎤⎟ ⎡1⎤ ⎛⎡          2⎤    